In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from minio import Minio
from minio.versioningconfig import VersioningConfig, ENABLED


In [2]:
# dump data
#
iris = load_iris(as_frame=True)
X, y = iris["data"], iris["target"]
data = pd.concat([X, y], axis="columns")
data.sample(100).to_csv("iris.csv", index=None)

In [3]:
url = "localhost:9000"
access_key = "minio"
secret_key = "miniostorage"
client = Minio(url, access_key=access_key, secret_key=secret_key, secure=False)


In [4]:
bucket_name = "raw-data"
object_name = "iris"
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)
    config = client.set_bucket_versioning(bucket_name, VersioningConfig(ENABLED))

# UPload
client.fput_object(bucket_name, object_name, "iris.csv")

In [5]:
# Download data

BUCKET_NAME, OBJECT_NAME = "raw-data", "iris"
# Minio 서버로부터 데이터를 다운로드
# stat_object 메소드로 객체의 메타데이터를 얻어오고 버전 ID를 획득
object_stat = client.stat_object(BUCKET_NAME, OBJECT_NAME)
data_version_id = object_stat.version_id
data_version_id

'd09c4eb6-e2e0-4729-a313-2d4626e62962'

In [6]:
client.fget_object(BUCKET_NAME, OBJECT_NAME, file_path="download_data.csv")
